In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# Any results you write to the current directory are saved as output.

In [ ]:
%matplotlib inline 

import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df= pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

**We will convert ObservationDate and Last Update to datetime since they are currently taken as object<br><br>
 The counts for 'Confirmed', 'Deaths' and 'Recovered' will be converted to int**

In [ ]:
df['ObservationDate'] = pd.to_datetime(df['ObservationDate'])
df['Last Update'] = pd.to_datetime(df['Last Update'])
df['Confirmed']=df['Confirmed'].astype('int')
df['Deaths']=df['Deaths'].astype('int')
df['Recovered']=df['Recovered'].astype('int')


**Since the final count of cases is present on 13th April 2020 we will create a separate dataframe for the same**

In [ ]:
from datetime import date
#df_update=df.loc[df.ObservationDate>pd.Timestamp(date(2020,4,12))]
df_update=df
df_update

In [ ]:
df_update.isnull().sum()

**Wherever Province is null, we replace it with the Country name and we group Mainland China and China together in China**

In [ ]:
df_update['Province/State']=df_update.apply(lambda x: x['Country/Region'] if pd.isnull(x['Province/State']) else x['Province/State'],axis=1)
df['Province/State']=df.apply(lambda x: x['Country/Region'] if pd.isnull(x['Province/State']) else x['Province/State'],axis=1)

In [ ]:
df_update['Country/Region']=df_update.apply(lambda x:'China' if x['Country/Region']=='Mainland China' else x['Country/Region'],axis=1)
df['Country/Region']=df.apply(lambda x:'China' if x['Country/Region']=='Mainland China' else x['Country/Region'],axis=1)

** We perform encoding of the Country to CountryID and Province to ProvinceID**

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df_update['ProvinceID'] = le.fit_transform(df_update['Province/State'])
df_update['CountryID']=le.fit_transform(df_update['Country/Region'])
df_update.head()

#  Printing the unique Countries and their count

**A total of 220 countries have been impacted uptill now.Note: All these countries may not necessarily have active cases now.**

In [ ]:
print(df['Country/Region'].unique())
print("\nNumber of unique countries in the dataset",len(df['Country/Region'].unique()))

## Add new columns to store new cases per day and growth rate

In [ ]:
new_df=df.groupby(['Country/Region','ObservationDate'],as_index=False).sum()
new_df['NewCases']=0
new_df['GrowthRate']=0


In [ ]:
country=new_df.loc[0]['Country/Region']
new_df['NewCases'][0]=new_df.loc[0]['Confirmed']
for i in range(1,len(new_df)):
    new_country=new_df.loc[i]['Country/Region']
    if(country == new_country):
        prev_day_total=new_df.loc[i-1]['Confirmed']
        today_total=new_df.loc[i]['Confirmed']
        if(today_total > 0):
            new_cases=today_total-prev_day_total
            daily_growth_rate = np.round((new_cases / today_total) * 100,2)
    else:
        new_cases=new_df['Confirmed'][i]
        daily_growth_rate = 0
        country = new_country
    new_df['NewCases'][i]=new_cases
    new_df['GrowthRate'][i]=daily_growth_rate
        


## Find Top 10 Countries based on Confirmed Cases

In [ ]:
from datetime import date 
from datetime import timedelta 
today = date.today() 
print("Today is: ", today) 
  
# Yesterday date 
yesterday = today - timedelta(days = 1) 
print("Yesterday was: ", yesterday) 

top10_df=new_df.loc[new_df['ObservationDate'] == yesterday].sort_values('Confirmed',ascending=False).head(10)
top10_country=top10_df['Country/Region'].values
print('Top 10 Countries:',top10_country)

next15_df=new_df.loc[new_df['ObservationDate'] == yesterday].sort_values('Confirmed',ascending=False).head(25).tail(15)
next15_country=next15_df['Country/Region'].values
print('Next Top 15 Countries:',next15_country)

## New Cases Trend for US, UK and India

In [ ]:
new_df.loc[new_df['Country/Region'].isin(['India','US','UK','Russia'])][['Country/Region','ObservationDate','NewCases' ]]\
    .pivot(index='ObservationDate',values='NewCases',columns='Country/Region')\
    .plot(kind='line',figsize=(30,10),title='New Cases Growth Trend for US, UK, Russia and India')


## New Cases Growth Rate for US, UK and India

In [ ]:
new_df.loc[new_df['Country/Region'].isin(['US','UK','India'])][['Country/Region','ObservationDate','GrowthRate' ]]\
    .pivot(index='ObservationDate',values='GrowthRate',columns='Country/Region')\
    .plot(kind='line',figsize=(30,10),title='New Cases Growth Rate for US, UK, Russia and India')


## New Cases and Growth Rate trend is declining in US and UK whereas it is rising in India

## New Cases Growth of Top 10 Countries

In [ ]:

new_df.loc[new_df['Country/Region'].isin(np.delete(top10_country,0))][['Country/Region','ObservationDate','Confirmed' ]]\
    .pivot(index='ObservationDate',values='Confirmed',columns='Country/Region')\
    .plot(kind='line',figsize=(20,10),title='New Cases Growth Trend for Top 10 Countries',marker='+',ylim=(0,500000),yticks=(1,10000,100000))


## New Cases Growth Trend for Next Top 15 Countries

In [ ]:
new_df.loc[new_df['Country/Region'].isin(next15_country)][['Country/Region','ObservationDate','NewCases' ]]\
    .pivot(index='ObservationDate',values='NewCases',columns='Country/Region')\
    .plot(kind='line',figsize=(30,20),title='New Cases Growth Trend for Next Top 15 Countries')


## Top 10 Countries Growth Rate

In [ ]:
new_df.loc[new_df['Country/Region'].isin(top10_country)][['Country/Region','ObservationDate','GrowthRate' ]]\
    .pivot(index='ObservationDate',values='GrowthRate',columns='Country/Region')\
    .plot(kind='line',figsize=(30,20),title='New Cases Growth Trend for Top 10 Countries')
